In [1]:
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2

# from spektral.layers import GraphConv
# GRaphConv is deprecated, use GCNConv or GCSConv instead
from spektral.datasets import mnist
from spektral.layers import GatedGraphConv
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.utils import normalized_laplacian
from spektral.layers import GCSConv  # as GraphConv
from spektral.layers import GCNConv  # as GraphConv

from spektral.utils.convolution import gcn_filter  # For GCNConv
from spektral.utils.convolution import normalized_adjacency  # For GCSConv

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

2023-11-19 14:39:35.335055: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 14:39:35.365251: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-19 14:39:35.365292: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-19 14:39:35.366401: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-19 14:39:35.371100: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-19 14:39:35.371438: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
# Parameters
l2_reg = 5e-4  # Regularization rate for l2
learning_rate = 1e-3  # Learning rate for SGD
batch_size = 32  # Batch size
epochs = 5  # Number of training epochs
es_patience = 200  # Patience fot early stopping

In [ ]:
# load the proximiity_matrix.csv
adj = np.loadtxt("adjacency_matrix.csv", delimiter=",")
print(adj.shape)

In [ ]:
# carrega os nós
nodes = np.loadtxt("vertices.csv", delimiter=",")
print(nodes.shape)

In [ ]:
# carrega os edges
edges = np.loadtxt("edges.csv", delimiter=",")
print(edges.shape)

In [ ]:
# carrega as distancias
y = np.loadtxt("distances.csv", delimiter=",")
print(y.shape)

In [ ]:
# cria um dataframe com os dados
df = pd.DataFrame({"source": edges[:, 0], "target": edges[:, 1], "weight": y})
df.head()

In [ ]:
# dimension of the target
n_out = 6

N = len(nodes)  # Number of nodes in the graph
F = 1  # Dimension of node features

In [ ]:
fltr = normalized_laplacian(adj)  # For GCNConv
# fltr = gcn_filter(adj) # For GCNConv
# fltr = normalized_adjacency(adj) # For GCSConv

https://graphneural.network/layers/convolution/#gcsconv

In [ ]:
# Prepare data
fltr = normalized_adjacency(adj)

# Model using GCSConv
X_in = Input(shape=(N, F))
# Pass A as a fixed tensor, otherwise Keras will complain about inputs of
# different rank.
A_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

graph_conv_1 = GCSConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([X_in, A_in])
graph_conv_2 = GCSConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([graph_conv_1, A_in])
flatten = Flatten()(graph_conv_2)
fc = Dense(512, activation="relu")(flatten)
output = Dense(n_out, activation="softmax")(fc)

# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)
model.summary()

In [ ]:
# train the GSCConv model
model.fit(
    [nodes, fltr],
    y,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=es_patience, restore_best_weights=True)],
)

https://graphneural.network/layers/convolution/#gcnconv

In [ ]:
# Prepare data
fltr = gcn_filter(adj)
# Model using GCNConv
X_in = Input(shape=(N, F))
# Pass A as a fixed tensor, otherwise Keras will complain about inputs of
# different rank.
A_in = Input(tensor=sp_matrix_to_sp_tensor(fltr))

graph_conv_1 = GCNConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([X_in, A_in])
graph_conv_2 = GCNConv(
    32, activation="elu", kernel_regularizer=l2(l2_reg), use_bias=True
)([graph_conv_1, A_in])
# removed the flatten layer because it is not necessary for GCNConv
fc = Dense(512, activation="relu")(graph_conv_2)
output = Dense(n_out, activation="softmax")(fc)


# Build model
model = Model(inputs=[X_in, A_in], outputs=output)
optimizer = Adam(lr=learning_rate)
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
)
model.summary()

In [ ]:
# train the GCNConv model

model.fit(
    [nodes, fltr],
    y,
    batch_size=batch_size,
    validation_split=0.1,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=es_patience, restore_best_weights=True)],
)